# Body parts

- Loss function: ArcFace
- SwimB
- closed set
- Segmentation
  - Full body - Simply combine all body parts
  - Fixed Weighted body parts - Combine all body parts with fixed Weights
  - Learnable Weighted body parts - Combine all body parts with Learnable Weights

In [2]:
import os
from pathlib import Path

In [3]:
from src.dataset import download_dataset
from src.utils import get_device
from src.pipeline import TurtleReIdPipeline
from src.model import PartsSwinBModel
from src.model import WeightedPartsSwinBModel
from src.model import LearnableWeightedPartsSwinBModel

In [4]:
# --- Pipeline parameters ---
device = get_device()
EMBEDDING_SIZE = 512


# --- Transform parameters ---
IMG_SIZE = 224


# --- Datasets parameters ---
DATA_DIR = './data/seaturtleid2022-subset'
train_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_train.csv")
eval_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_valid.csv")
test_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_test.csv")
datasets_paths = {
    'train_csv_path': train_csv_path,
    'eval_csv_path': eval_csv_path,
    'test_csv_path': test_csv_path,
}

paths = download_dataset()
img_dir=paths['images_path']

annotations_path= f"{DATA_DIR}/annotations.json"


# --- Training parameters ---
BATCH_SIZE = 10
LEARNING_RATE = 1e-4
EPOCHS = 10

model_dir = './models'
Path(model_dir).mkdir(parents=True, exist_ok=True)


train_configs = {
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'epochs': EPOCHS,
    # 'model_save_path': model_save_path,
    'k': 1
}

Dataset downloaded and extracted to: /Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4


### Whole turtle body

In [4]:
turtle_model = PartsSwinBModel(embedding_size=EMBEDDING_SIZE)
turtle_pipeline = TurtleReIdPipeline(device, EMBEDDING_SIZE, model=turtle_model)
turtle_pipeline.set_transforms(IMG_SIZE)
turtle_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [6]:
# --- Training Loop on All Parts ---
turtle_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_combination_closed_arcface_swimb.pth'
turtle_configs['model_save_path'] = model_save_path
turtle_configs['target_part'] = 'parts'  # all parts (head, flipper, body)
turtle_pipeline.train(turtle_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/10, Loss: 16.3362, Eval Accuracy: 45.12%
Saved best model.


Epoch 2/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 2/10, Loss: 13.8179, Eval Accuracy: 63.41%
Saved best model.


Epoch 3/10:   0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 0.4455, Eval Accuracy: 80.49%


Epoch 9/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.3164, Eval Accuracy: 80.49%


Epoch 10/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.2278, Eval Accuracy: 79.27%
Finished Training. Best Test Accuracy: 84.15%


### Whole Turtle body with Fixed weights

In [4]:
fixed_model = WeightedPartsSwinBModel(
    embedding_size=EMBEDDING_SIZE, part_weights=[1, 1.5, 2])
fixed_pipeline = TurtleReIdPipeline(device, EMBEDDING_SIZE, model=fixed_model)
fixed_pipeline.set_transforms(IMG_SIZE)
fixed_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [5]:
# --- Training Loop on FIXED WEIGHTs ---
fixed_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_fixed_combination_closed_arcface_swimb.pth'
fixed_configs['model_save_path'] = model_save_path
fixed_configs['target_part'] = 'parts'  # all parts (head, flipper, body)
fixed_pipeline.train(fixed_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/10, Loss: 16.8517, Eval Accuracy: 50.00%
Saved best model.


Epoch 2/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 2/10, Loss: 14.1906, Eval Accuracy: 58.54%
Saved best model.


Epoch 3/10:   0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 0.8690, Eval Accuracy: 81.71%
Saved best model.


Epoch 9/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.4747, Eval Accuracy: 79.27%


Epoch 10/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.6709, Eval Accuracy: 81.71%
Finished Training. Best Test Accuracy: 81.71%


### Whole Turtle body with Trainable weights

In [ ]:
learnable_model = LearnableWeightedPartsSwinBModel(
    embedding_size=EMBEDDING_SIZE, per_dimension=False) 
learnable_pipeline = TurtleReIdPipeline(
    device, EMBEDDING_SIZE, model=learnable_model)
learnable_pipeline.set_transforms(IMG_SIZE)
learnable_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [5]:
# --- Training Loop on LEARNABLE WEIGHTs ---
learnable_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_learnable_combination_closed_arcface_swimb.pth'
learnable_configs['model_save_path'] = model_save_path
learnable_configs['target_part'] = 'parts'  # trainable learnable parts
learnable_pipeline.train(learnable_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/10, Loss: 16.9537, Eval Accuracy: 41.46%
Saved best model.


Epoch 2/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 2/10, Loss: 14.4032, Eval Accuracy: 68.29%
Saved best model.


Epoch 3/10:   0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 0.7554, Eval Accuracy: 75.61%


Epoch 9/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.4872, Eval Accuracy: 76.83%


Epoch 10/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.3831, Eval Accuracy: 78.05%
Saved best model.
Finished Training. Best Test Accuracy: 78.05%


### Whole Turtle body with Trainable weights per demention

In [5]:
dim_learnable_model = LearnableWeightedPartsSwinBModel(
    embedding_size=EMBEDDING_SIZE, per_dimension=True) 
dim_learnable_pipeline = TurtleReIdPipeline(
    device, EMBEDDING_SIZE, model=dim_learnable_model)
dim_learnable_pipeline.set_transforms(IMG_SIZE)
dim_learnable_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [6]:
# --- Training Loop on LEARNABLE PER-DIMENSION WEIGHTs ---
dim_learnable_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_dim_learnable_combination_closed_arcface_swimb.pth'
dim_learnable_configs['model_save_path'] = model_save_path
dim_learnable_configs['target_part'] = 'parts'  # trainable learnable parts
dim_learnable_pipeline.train(dim_learnable_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/10, Loss: 16.6349, Eval Accuracy: 53.66%
Saved best model.


Epoch 2/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 2/10, Loss: 14.0478, Eval Accuracy: 54.88%
Saved best model.


Epoch 3/10:   0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 0.6315, Eval Accuracy: 74.39%


Epoch 9/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.2543, Eval Accuracy: 73.17%


Epoch 10/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.3204, Eval Accuracy: 70.73%
Finished Training. Best Test Accuracy: 75.61%
